# Project 6 - Pipleline 1 - Download

In [1]:
import os
os.chdir('../../')

In [2]:
import yaml
import pandas as pd
import lib.database_module as db
import lib.encoding_module as enc
import lib.wiki_module as wiki

## Acquire: Get categories to query from a yaml file

In [5]:
with open('config/categories.yml') as yml_file:
    categories = (yaml.load(yml_file))

In [6]:
categories

{'categories': ['Influenza']}

## Acquire: Query Wikipedia by category for pages

In [10]:
responses = []
for c in categories['categories']:
    response = wiki.query_category(c)
    responses.append(response)

In [15]:
responses[0]['categoryid']

u'2932559'

In [11]:
pd.DataFrame(responses[0]['pages']).head(2)

,pageid,title
0,19572217,Influenza
1,50576530,Timeline of influenza


## Acquire: Write category info to database

In [16]:
db.create_or_update_category_in_database('Influenza',2932559)

Connected to server joshuacook.me.


'ERROR:  column "influenza" does not exist\nLINE 3:         (category_id, category_name) VALUES (Influenza, \'293...\n                                                     ^\n'

## Acquire: Query Wikipedia by page id for content

In [19]:
wiki.query_page('19572217','Influenza')['text']

u'Influenza, commonly known as "the flu", is an infectious disease caused by an influenza virus. Symptoms can be mild to severe. The most common symptoms include: a high fever, runny nose, sore throat, muscle pains, headache, coughing, and feeling tired. These symptoms typically begin two days after exposure to the virus and most last less than a week. The cough, however, may last for more than two weeks. In children, there may be nausea and vomiting, but these are not common in adults. Nausea and vomiting occur more commonly in the unrelated infection gastroenteritis, which is sometimes inaccurately referred to as "stomach flu" or "24-hour flu". Complications of influenza may include viral pneumonia, secondary bacterial pneumonia, sinus infections, and worsening of previous health problems such as asthma or heart failure. Three types of influenza viruses affect people, called Type A, Type B, and Type C. Usually, the virus is spread through the air from coughs or sneezes. This is belie

In [8]:
response = wiki.query_category("category")
for page in response['pages']:
    db.create_or_update_page_in_database(page['pageid'],2932559,page['title'],
                                         wiki.query_page(str(page['pageid']),page['title'])['text'])

## Identify: Provide a detailed analysis of the database schema in your report

In [3]:
con, cur = db.connect_to_postgres()
cur.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print cur.fetchall()
cur.close()
con.close()

Connected to server joshuacook.me.
[('page_vec',), ('category',), ('page',), ('page_cate',), ('cate_vec',)]


In [6]:
con, cur = db.connect_to_postgres()
cur.execute("""select * from page;""")
print [desc[0] for desc in cur.description]
cur.close()
con.close()

Connected to server joshuacook.me.
['page_id', 'title', 'created_at', 'page']


In [7]:
con, cur = db.connect_to_postgres()
cur.execute("""select * from page_cate;""")
print [desc[0] for desc in cur.description]
cur.close()
con.close()

Connected to server joshuacook.me.
['page_id', 'category_id']


In [8]:
con, cur = db.connect_to_postgres()
cur.execute("""select * from category;""")
print [desc[0] for desc in cur.description]
cur.close()
con.close()

Connected to server joshuacook.me.
['category_id', 'category_name']


## Identify: Provide a detailed analysis of the document encoding methods being used in your report

For the encoding, I took all of the pages and cleaned them up (removing punctuations, lowercase). Then, ran all of the pages though latent semantic analysis (LSA) with tfidfVectorizer, Normalize the data, then reduced dimentionality with truncated SVD. For search query I set the params of SVD to n_components 500 and for perdict category to 17. I used 17 because we had 17 categories which worked very well.